In [18]:
import pandas as pd

import os
import glob

In [64]:
aux_path = glob.glob('../../sadie/airr/data/germlines/**/*.aux',recursive=True)
their_aux_path = glob.glob('/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/**.aux')
their_aux_path

['/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/mouse_gl.aux',
 '/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/rabbit_gl.aux',
 '/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/rhesus_monkey_gl.aux',
 '/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/human_gl.aux',
 '/Users/jordanwillis/repos/personal/sadie/tests/reference/fixtures/igblast_aux/rat_gl.aux']

In [89]:
my_aux = []
for file in aux_path:
    df = pd.read_csv(file,skip_blank_lines=True,delimiter='\t',header=None,names=['gene','reading_frame','segment','cdr3_end','left_over'])
    df.insert(0,'common',os.path.basename(file).split('_')[0])
    df.insert(1,'db_type',file.split('/')[-4] )
    my_aux.append(df)
my_aux = pd.concat(my_aux).reset_index(drop=True).set_index(['common','db_type','gene'])

In [90]:
their_aux = []
for file in their_aux_path:
    df = pd.read_csv(file,skip_blank_lines=True,delim_whitespace=True,skiprows=2,header=None,names=['gene','reading_frame','segment','cdr3_end','left_over'])
    df.insert(0,'common',os.path.basename(file).split('_')[0])
    df.insert(1,'db_type','imgt')
    their_aux.append(df)
their_aux = pd.concat(their_aux).reset_index(drop=True).set_index(['common','db_type','gene'])

In [95]:
common_index = my_aux.index.intersection(their_aux.index)

In [121]:
i_dont_have = their_aux.index.difference(my_aux.index)

In [123]:
they_dont_have = my_aux.index.difference(their_aux.index)

In [124]:
they_dont_have

MultiIndex([( 'alpaca',   'imgt', 'IGHJ1*01'),
            ( 'alpaca',   'imgt', 'IGHJ2*01'),
            ( 'alpaca',   'imgt', 'IGHJ3*01'),
            ( 'alpaca',   'imgt', 'IGHJ4*01'),
            ( 'alpaca',   'imgt', 'IGHJ5*01'),
            ( 'alpaca',   'imgt', 'IGHJ6*01'),
            ( 'alpaca',   'imgt', 'IGHJ7*01'),
            (    'cat', 'custom', 'IGKJ1*01'),
            (    'cat', 'custom', 'IGKJ2*01'),
            (    'cat', 'custom', 'IGKJ3*01'),
            ...
            ('macaque',   'imgt', 'IGKJ2*01'),
            ('macaque',   'imgt', 'IGKJ3*01'),
            ('macaque',   'imgt', 'IGKJ4*01'),
            ('macaque',   'imgt', 'IGKJ5*01'),
            ('macaque',   'imgt', 'IGLJ1*01'),
            ('macaque',   'imgt', 'IGLJ2*01'),
            ('macaque',   'imgt', 'IGLJ3*01'),
            ('macaque',   'imgt', 'IGLJ4*01'),
            ('macaque',   'imgt', 'IGLJ5*01'),
            ('macaque',   'imgt', 'IGLJ6*01')],
           names=['common', 'db_type', 'gen

In [100]:
their_aux_common_index = their_aux.loc[common_index]

In [101]:
my_aux_common_index = my_aux.loc[common_index]

In [120]:
known_aux_exceptions = {("mouse", "imgt", "IGLJ4*01"):'igblast has wrong number of c-term remaining',
                        ('rabbit', 'imgt', 'IGHJ3*02'):'igblast has wrong reading frame'}
for index in my_aux_common_index.index:
    try:
        pd._testing.assert_series_equal(their_aux_common_index.loc[index],my_aux_common_index.loc[index])
    except AssertionError:
        if index in known_aux_exceptions.keys():
            print(index,'is exception')
            continue
        else:
            pd._testing.assert_series_equal(their_aux_common_index.loc[index],my_aux_common_index.loc[index],obj=index)

('mouse', 'imgt', 'IGLJ4*01') is exception
('rabbit', 'imgt', 'IGHJ3*02') is exception


In [116]:
index

('rabbit', 'imgt', 'IGHJ3*02')

In [107]:
their_aux_common_index.loc[index]

reading_frame     1
segment          JL
cdr3_end          6
left_over         1
Name: (mouse, imgt, IGLJ4*01), dtype: object

In [111]:
my_aux_common_index.loc[index] == their_aux_common_index.loc[index])

reading_frame     True
segment           True
cdr3_end          True
left_over        False
Name: (mouse, imgt, IGLJ4*01), dtype: bool

In [110]:
my_aux.loc[common_index] == their_aux.loc[common_index]

reading_frame  segment  cdr3_end  left_over
common db_type gene                                                 
mouse  imgt    IGHJ1*01           True     True      True       True
               IGHJ1*02           True     True      True       True
               IGHJ1*03           True     True      True       True
               IGHJ2*01           True     True      True       True
               IGHJ2*02           True     True      True       True
...                                ...      ...       ...        ...
rat    imgt    IGKJ5*01           True     True      True       True
               IGLJ1*01           True     True      True       True
               IGLJ2*01           True     True      True       True
               IGLJ3*01           True     True      True       True
               IGLJ4*01           True     True      True       True

[103 rows x 4 columns]

In [99]:
pd._testing.assert_frame_equal(my_aux.loc[common_index],their_aux.loc[common_index],check_like=True)

AssertionError: DataFrame.iloc[:, 0] (column name="reading_frame") are different

DataFrame.iloc[:, 0] (column name="reading_frame") values are different (0.97087 %)
[index]: [(mouse, imgt, IGHJ1*01), (mouse, imgt, IGHJ1*02), (mouse, imgt, IGHJ1*03), (mouse, imgt, IGHJ2*01), (mouse, imgt, IGHJ2*02), (mouse, imgt, IGHJ2*03), (mouse, imgt, IGHJ3*01), (mouse, imgt, IGHJ3*02), (mouse, imgt, IGHJ4*01), (mouse, imgt, IGKJ1*01), (mouse, imgt, IGKJ1*02), (mouse, imgt, IGKJ2*01), (mouse, imgt, IGKJ2*02), (mouse, imgt, IGKJ2*03), (mouse, imgt, IGKJ3*01), (mouse, imgt, IGKJ3*02), (mouse, imgt, IGKJ4*01), (mouse, imgt, IGKJ4*02), (mouse, imgt, IGKJ5*01), (mouse, imgt, IGLJ1*01), (mouse, imgt, IGLJ2*01), (mouse, imgt, IGLJ3*01), (mouse, imgt, IGLJ4*01), (rabbit, imgt, IGHJ1*01), (rabbit, imgt, IGHJ2*01), (rabbit, imgt, IGHJ2*02), (rabbit, imgt, IGHJ3*01), (rabbit, imgt, IGHJ3*02), (rabbit, imgt, IGHJ4*01), (rabbit, imgt, IGHJ4*02), (rabbit, imgt, IGHJ5*01), (rabbit, imgt, IGHJ5*02), (rabbit, imgt, 